# Bring Your Own Data (BYOD) Template

This notebook provides a template for users to load their own dataset and convert it to OpenVisus IDX format. The data could be of any format as long as they can be loaded into thememory as numpy arrays.

## Step 1: Import Libraries

In [ ]:
import numpy as np
import OpenVisus as ov
import xarray as xr


## Step 2: Set Your File Paths and Parameters

Update the variables below to match your dataset.

In [ ]:
# Example: local_nc = 'path/to/your/file.nc'
local_nc = 'your_file.nc'  # Path to your data
field_name = 'your_field_name'  # Name for the IDX field
idx_name = 'your_idx_name'      # Name for the IDX file
dims = [h, w]             # Update with your data dimensions
timesteps=[]                  # Update with your year or time info

### Step 3: Load Your Data

In [ ]:
ds = xr.open_dataset(local_nc)
data = ds['your_variable'].values  # Replace 'your_variable' with your variable name
print(data.shape)

### Step 4: Create IDX Dataset and Write Data

In [ ]:
field = ov.Field(field_name, 'float32')
os.makedirs('idx_dir', exist_ok=True)
idx = ov.CreateIdx(
    url=f'idx_dir/{idx_name}.idx',
    fields=[field],
    time=[TIME_START, TIME_END, 'time_%d/'],
    arco='modvisus',
    dims=dims,
)

for day in range(data.shape[0]):
    idx.write(data[day], time=TIME_OF_INTEREST, field=field)

ds.close()

### Step 5: (Optional) Compress IDX Dataset

In [ ]:
idx.compressDataset(compression='zip')

### Step 6: Verify the Data

In [ ]:
db = ov.LoadDataset(f'idx_dir/{idx_name}.idx')
data = db.read()
import matplotlib.pyplot as plt
plt.imshow(data, cmap='viridis', origin='lower')
plt.colorbar(label='Your Variable Units')
plt.show()